In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from os import listdir
from os.path import getsize
from pathlib import Path

In [ ]:
output_path = 'data/output'
filters_path = "data/filters"

In [ ]:
files = listdir(output_path)

In [ ]:
sizes = []
files_under_5mb = []
files_under_5mb_size = []
for file in files:
    sizes.append(getsize(f"{output_path}/{file}"))
    if getsize(f"{output_path}/{file}") < 5000000:
        files_under_5mb.append(file)
        files_under_5mb_size.append(getsize(f"{output_path}/{file}"))

In [ ]:
print(f"Total Size: {sum(sizes)/1000000000} GB")
print(f"Count: {len(files_under_5mb)}")
print(f"Files under 5mb Total Size: {sum(files_under_5mb_size)/1000000000} GB")

In [ ]:
def create_filter(file):
    default_checked = {"geo": "Canada", "uom": "Tonnes", "type_of_structure": "Total residential", "type_of_work": "Types of work, total"}
    col_names = {"geo": "Geography", "uom": "Unit of Measurement"}
    ignore_cols = ["ref_date", "dguid", "uom_id", "scalar_factor", "scalar_id", "vector", "coordinate", "value", "symbol", "terminated", "decimals", "status"]
    unique_filter_list = []
    
    dt = pd.read_json(f"data/output/{file}")

    for col in dt.columns:
        try: 

            if col not in ignore_cols:

                filter_list = []
                for row in dt[col]:
                    filter_list.append(row)
                unique_filter = np.unique(filter_list).tolist()
                unique_filter_dict = []

                for value in unique_filter:
                    for index, (col_checked, filter_checked) in enumerate(default_checked.items()):
                        if col == col_checked and value == filter_checked:
                            unique_filter_dict.append({"name": value, "filter":True})
                        elif {"name": value, "filter":False} not in unique_filter_dict and {"name": value, "filter":True} not in unique_filter_dict:
                            unique_filter_dict.append({"name": value, "filter":False})



                if len([item for item in unique_filter_dict if item['filter'] == True]) == 0:
                    for index, value in enumerate(unique_filter_dict):
                        if index == 0:
                            value['filter'] = True




                for index, (col_original, name) in enumerate(col_names.items()):
                    if col == col_original:
                        friendly_name = name
                        break;
                    else:
                        friendly_name = str.replace(col.title(), "_", " ")

                if col == 'geo':
                    pivot_value = True
                else:
                    pivot_value = False

                unique_filter_list.append({"col": col, "name": friendly_name, "pivot": pivot_value, "filters": unique_filter_dict})
    
        except Exception as e:
            print(e)

    with open(f'data/filters/{Path(file).stem}-filters.json', 'w') as f:
        json.dump(unique_filter_list, f)

In [ ]:
for file in tqdm(files_under_5mb):
    create_filter(file)